### Learn Azure in Hong Kong | Multimodel Multimodal and Multiagent with Azure

Cantonese series talking about everything related to AI on Azure.  
廣東話系列講述與 Azure AI 相關的一切。  
广东话讲述与 Azure AI 相关的一切。

Addressing real-world scenarios with AI models requires a diverse approach. Each model may offer unique capabilities, and numerous types of source files, such as text, image, and video, need to be considered.

In this session, we will demonstrate practical applications of multimodal and multiagent sample using Azure AI services. Join us for this Microsoft Reactor session to learn how to manage these varied scenarios and benefit from insights on actual case experiences.

處理實際場景中的 AI 模型需要多元化的方法。每個模型可能提供獨特的功能，且需要考慮多種類型的源文件，如文本、圖像和影片。

在本次分享中，我們將演示使用 Azure AI 服務的多模式和多代理範例的實際應用。加入我們的 Microsoft Reactor 會議，了解如何管理這些不同的場景，並從對實際案例體驗的見解中受益。

解决现实场景中的 AI 模型问题需要多样化的方法。每个模型可能提供独特的能力，并且需要考虑多种类型的源文件，如文本、图像和视频。

在本次分享中，我们将展示如何使用 Azure AI 服务进行多模态和多代理的实际应用示例。加入我们，了解如何管理这些多样化的场景，并从实际案例经验中获益。

Event time: 28 Feb 2025 19:30-20:30  
Event Type: Microsoft Reactor online live  
Event URL: https://developer.microsoft.com/en-us/reactor/events/25041/


In [134]:
# Provide the information of database connection
db_conn = {
    'host': 'localhost',  # Database server hostname or IP address
    'port': 3306,       # Database connection port number
    'user': 'root',       # Database username
    'password': '123456', # Database password
    'database': 'demo_db' # Name of database to connect to
}

In [135]:
# Provide the database schema for Text2SQL model
db_schema = """
CREATE TABLE `customers` (
  `customerNumber` int NOT NULL,
  `customerName` varchar(50) NOT NULL,
  `contactLastName` varchar(50) NOT NULL,
  `contactFirstName` varchar(50) NOT NULL,
  `phone` varchar(50) NOT NULL,
  `addressLine1` varchar(50) NOT NULL,
  `addressLine2` varchar(50) DEFAULT NULL,
  `city` varchar(50) NOT NULL,
  `state` varchar(50) DEFAULT NULL,
  `postalCode` varchar(15) DEFAULT NULL,
  `country` varchar(50) NOT NULL,
  `salesRepEmployeeNumber` int DEFAULT NULL,
  `creditLimit` decimal(10,2) DEFAULT NULL,
  PRIMARY KEY (`customerNumber`),
  KEY `salesRepEmployeeNumber` (`salesRepEmployeeNumber`),
  CONSTRAINT `customers_ibfk_1` FOREIGN KEY (`salesRepEmployeeNumber`) REFERENCES `employees` (`employeeNumber`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `employees` (
  `employeeNumber` int NOT NULL,
  `lastName` varchar(50) NOT NULL,
  `firstName` varchar(50) NOT NULL,
  `extension` varchar(10) NOT NULL,
  `email` varchar(100) NOT NULL,
  `officeCode` varchar(10) NOT NULL,
  `reportsTo` int DEFAULT NULL,
  `jobTitle` varchar(50) NOT NULL,
  PRIMARY KEY (`employeeNumber`),
  KEY `reportsTo` (`reportsTo`),
  KEY `officeCode` (`officeCode`),
  CONSTRAINT `employees_ibfk_1` FOREIGN KEY (`reportsTo`) REFERENCES `employees` (`employeeNumber`),
  CONSTRAINT `employees_ibfk_2` FOREIGN KEY (`officeCode`) REFERENCES `offices` (`officeCode`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `offices` (
  `officeCode` varchar(10) NOT NULL,
  `city` varchar(50) NOT NULL,
  `phone` varchar(50) NOT NULL,
  `addressLine1` varchar(50) NOT NULL,
  `addressLine2` varchar(50) DEFAULT NULL,
  `state` varchar(50) DEFAULT NULL,
  `country` varchar(50) NOT NULL,
  `postalCode` varchar(15) NOT NULL,
  `territory` varchar(10) NOT NULL,
  PRIMARY KEY (`officeCode`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `orderdetails` (
  `orderNumber` int NOT NULL,
  `productCode` varchar(15) NOT NULL,
  `quantityOrdered` int NOT NULL,
  `priceEach` decimal(10,2) NOT NULL,
  `orderLineNumber` smallint NOT NULL,
  PRIMARY KEY (`orderNumber`,`productCode`),
  KEY `productCode` (`productCode`),
  CONSTRAINT `orderdetails_ibfk_1` FOREIGN KEY (`orderNumber`) REFERENCES `orders` (`orderNumber`),
  CONSTRAINT `orderdetails_ibfk_2` FOREIGN KEY (`productCode`) REFERENCES `products` (`productCode`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `orders` (
  `orderNumber` int NOT NULL,
  `orderDate` date NOT NULL,
  `requiredDate` date NOT NULL,
  `shippedDate` date DEFAULT NULL,
  `status` varchar(15) NOT NULL,
  `comments` text,
  `customerNumber` int NOT NULL,
  PRIMARY KEY (`orderNumber`),
  KEY `customerNumber` (`customerNumber`),
  CONSTRAINT `orders_ibfk_1` FOREIGN KEY (`customerNumber`) REFERENCES `customers` (`customerNumber`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `payments` (
  `customerNumber` int NOT NULL,
  `checkNumber` varchar(50) NOT NULL,
  `paymentDate` date NOT NULL,
  `amount` decimal(10,2) NOT NULL,
  PRIMARY KEY (`customerNumber`,`checkNumber`),
  CONSTRAINT `payments_ibfk_1` FOREIGN KEY (`customerNumber`) REFERENCES `customers` (`customerNumber`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `productlines` (
  `productLine` varchar(50) NOT NULL,
  `textDescription` varchar(4000) DEFAULT NULL,
  `htmlDescription` mediumtext,
  `image` mediumblob,
  PRIMARY KEY (`productLine`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

CREATE TABLE `products` (
  `productCode` varchar(15) NOT NULL,
  `productName` varchar(70) NOT NULL,
  `productLine` varchar(50) NOT NULL,
  `productScale` varchar(10) NOT NULL,
  `productVendor` varchar(50) NOT NULL,
  `productDescription` text NOT NULL,
  `quantityInStock` smallint NOT NULL,
  `buyPrice` decimal(10,2) NOT NULL,
  `MSRP` decimal(10,2) NOT NULL,
  PRIMARY KEY (`productCode`),
  KEY `productLine` (`productLine`),
  CONSTRAINT `products_ibfk_1` FOREIGN KEY (`productLine`) REFERENCES `productlines` (`productLine`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

"""

### Ask the question you would like to know from the database

In [136]:
question = "Which employee sell the most product? Give me the name of the employee"

### Using Text2SQL LLM Model to generate SQL Query according to the question and DB schema

In [ ]:
# The prompt for TEXT2SQL model
text2sql_prompt = f"""### Task
Generate a SQL query to answer [QUESTION]"""+question+"""[/QUESTION]

### Database Schema
"""+db_schema+"""

### Instructions
As an AI chatbot specialized in SQL queries for MySQL databases, your task is to generate a syntactically correct MySQL query to retrieve information from the provided database schema. Follow these guidelines:
- Utilize your expertise in MySQL to craft efficient queries that return relevant data.
- Use the schema provided to identify the appropriate tables and columns for your query.
- If the question involves conditions such as "today" or "current", utilize the GETDATE() function to retrieve the current date and time.
- Employ joins between tables if necessary to gather relevant information.
- Query for at most [top_k] results using the OFFSET FETCH clause for performance optimization. You can specify the desired value for [top_k] when calling the function.
- Select only the columns necessary to answer the question. Enclose each column name in square brackets ([]) to denote them as delimited identifiers.
- Avoid querying for all columns from a table to enhance efficiency.
- If the question cannot be answered with the available database schema, return 'I do not know'.
- ***When ordering results, handle NULL values appropriately and DO NOT using the 'NULLS LAST' syntax.***

**Missing Schema Handling:**
If the database schema information is missing, please provide it for query generation. Otherwise, this process will return 'Missing database schema'.

### Answer
Given the database schema, here is the SQL query that [QUESTION]"""+question+"""[/QUESTION]
[SQL]"""

print(text2sql_prompt)

In [138]:
%pip install requests
%pip install pymysql


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [139]:
import requests
import json

# Set the URL for the local Ollama server (adjust the port if needed)
url = "http://localhost:11434/api/generate"

# Prepare the payload with the model name and prompt
payload = {
    "model": "pxlksr/defog_sqlcoder-7b-2:F16",
    "prompt": text2sql_prompt,  # using the variable from cell index 4
    "stream": False
}

headers = {
    "Content-Type": "application/json"
}

# Send a POST request to the Ollama endpoint
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    # Save the output response in string format into a variable
    response_str = json.dumps(result)
    print("Generated SQL Query:")
    # print(result['response'])
    output_query = result['response']
    print(output_query)
else:
    print("Request failed with status code:", response.status_code)
    print(response.text)

Generated SQL Query:
 SELECT e.firstName, e.lastName FROM employees e JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber WHERE c.customerNumber IN (SELECT customerNumber FROM orders) ORDER BY COUNT(c.customerNumber) DESC LIMIT 1;


In [151]:
import pymysql  # Import library for MySQL database interaction
import json    # Import library for JSON data processing

def main(sql_command,db_conn):
    """
    Connect to MySQL database and execute the given SQL command.

    Parameters:
    json_data (str): JSON string containing database connection information.
    sql_command (str): SQL query command to execute.

    Returns:
    dict: Dictionary containing query results or error message.
    """
    try:
        # Parse JSON string into Python dictionary
        #config = json.loads(json_data)['database']
        
        # Establish database connection using configuration info
        connection = pymysql.connect(
            host=db_conn['host'],        # Database server hostname or IP address
            port=db_conn['port'],        # Database connection port number
            user=db_conn['user'],        # Database username
            password=db_conn['password'], # Database password
            database=db_conn['database'] # Name of database to connect to
        )
        
        try:
            with connection.cursor() as cursor:  # Get database cursor using context manager
                # Execute SQL command
                cursor.execute(sql_command)
                
                # Get all query results
                result = cursor.fetchall()
                
                output_result = str(result)
                # Convert results to JSON string format
                #result_str = json.dumps(result, ensure_ascii=False)
                
                # Return dictionary containing query results
                return {
                    "result": output_result
                }
        
        finally:
            # Ensure database connection is closed to free resources
            connection.close()
    
    except Exception as e:
        # Catch and return any exception information
        return {
            "result": f"Error: {str(e)}"
        }

output_result = main(output_query,db_conn)
print(output_result)

{'result': "Error: (3029, 'Expression #1 of ORDER BY contains aggregate function and applies to the result of a non-aggregated query')"}


In [ ]:
AZUREAI_MODEL="gpt-4o"
AZUREAI_ENDPOINT_URL="YOUR_AZURE_OPENAI_URL"
AZUREAI_ENDPOINT_KEY="YOUR_API_KEY"

In [143]:
query_translate_prompt = f"""
```xml
<instructions>
You are tasked with fine-tuning an SQL query based on an error message provided. Follow these steps to complete the task:

1. Carefully analyze the error message to understand the issue. Identify the specific part of the SQL query that is causing the error. Common issues may include syntax errors, missing or incorrect keywords, invalid column names, or mismatched data types.

2. Review the SQL query provided alongside the error message. Ensure you understand the query's intent and the expected outcome.

3. Cross-reference the error message with the SQL query to pinpoint the exact problem. Use your knowledge of SQL syntax, functions, and best practices to identify potential fixes.

4. Rewrite or adjust the SQL query to resolve the issue highlighted by the error message. Ensure that the updated query adheres to SQL standards and achieves the intended functionality.

5. Test the revised query in your mind or through logical reasoning to ensure it resolves the error and produces the desired result.

6. Provide the corrected SQL query as the output. Ensure the output is clean, concise, and free of any unnecessary comments or explanations.

7. Do not include any XML tags in the output. The output should only contain the corrected SQL query.

</instructions>

<examples>
<example>
<error_message>
Syntax error near 'FROM' at line 2.
</error_message>
<sql_query>
SELECT name age
FROM users;
</sql_query>
<corrected_query>
SELECT name, age
FROM users;
</corrected_query>
</example>

<example>
<error_message>
Column 'user_id' does not exist in table 'orders'.
</error_message>
<sql_query>
SELECT user_id, order_date
FROM orders;
</sql_query>
<corrected_query>
SELECT customer_id, order_date
FROM orders;
</corrected_query>
</example>

<example>
<error_message>
Data type mismatch in WHERE clause: expected integer but found string.
</error_message>
<sql_query>
SELECT *
FROM products
WHERE product_id = '123';
</sql_query>
<corrected_query>
SELECT *
FROM products
WHERE product_id = 123;
</corrected_query>
</example>
</examples>


<error_message>
"""+str(output_result)+"""
</error_message>
<sql_query>
"""+str(output_query)+"""
</sql_query>
```
"""

print(query_translate_prompt)


```xml
<instructions>
You are tasked with fine-tuning an SQL query based on an error message provided. Follow these steps to complete the task:

1. Carefully analyze the error message to understand the issue. Identify the specific part of the SQL query that is causing the error. Common issues may include syntax errors, missing or incorrect keywords, invalid column names, or mismatched data types.

2. Review the SQL query provided alongside the error message. Ensure you understand the query's intent and the expected outcome.

3. Cross-reference the error message with the SQL query to pinpoint the exact problem. Use your knowledge of SQL syntax, functions, and best practices to identify potential fixes.

4. Rewrite or adjust the SQL query to resolve the issue highlighted by the error message. Ensure that the updated query adheres to SQL standards and achieves the intended functionality.

5. Test the revised query in your mind or through logical reasoning to ensure it resolves the error 

In [147]:
import requests
import json

url = AZUREAI_ENDPOINT_URL

payload = json.dumps({
  "messages": [
      {
          "role": "system",
          "content": query_translate_prompt
      }
  ],
  "frequency_penalty": 0,
  "presence_penalty": 0,
  "max_tokens": 4000,
  "seed": 42,
  "stop": "<|endoftext|>",
  "stream": False,
  "temperature": 0,
  "top_p": 1,
  "response_format": {
    "type": "text"
  }
})
headers = {
  'Content-Type': 'application/json',
  'api-key': AZUREAI_ENDPOINT_KEY
}

response = requests.request("POST", url, headers=headers, data=payload)
resp = json.loads(response.text)
new_query = resp['choices'][0]['message']['content']

print(resp['choices'][0]['message']['content'])


SELECT e.firstName, e.lastName, COUNT(c.customerNumber) AS customerCount
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE c.customerNumber IN (SELECT customerNumber FROM orders)
GROUP BY e.firstName, e.lastName
ORDER BY customerCount DESC
LIMIT 1;


In [148]:
import pymysql  # Import library for MySQL database interaction
import json    # Import library for JSON data processing

def main(sql_command,db_conn):
    """
    Connect to MySQL database and execute the given SQL command.

    Parameters:
    json_data (str): JSON string containing database connection information.
    sql_command (str): SQL query command to execute.

    Returns:
    dict: Dictionary containing query results or error message.
    """
    try:
        # Parse JSON string into Python dictionary
        #config = json.loads(json_data)['database']
        
        # Establish database connection using configuration info
        connection = pymysql.connect(
            host=db_conn['host'],        # Database server hostname or IP address
            port=db_conn['port'],        # Database connection port number
            user=db_conn['user'],        # Database username
            password=db_conn['password'], # Database password
            database=db_conn['database'] # Name of database to connect to
        )
        
        try:
            with connection.cursor() as cursor:  # Get database cursor using context manager
                # Execute SQL command
                cursor.execute(sql_command)
                
                # Get all query results
                result = cursor.fetchall()
                
                _output_result = str(result)
                # Convert results to JSON string format
                #result_str = json.dumps(result, ensure_ascii=False)
                
                # Return dictionary containing query results
                return {
                    "result": str(_output_result)
                }
        
        finally:
            # Ensure database connection is closed to free resources
            connection.close()
    
    except Exception as e:
        # Catch and return any exception information
        return {
            "result": f"Error: {str(e)}"
        }

output_result = main(new_query,db_conn)['result']
print(output_result)

(('Pamela', 'Castillo', 10),)


In [149]:
system_prompt = f"""
```xml
<instructions>
You are tasked with answering a question based on the provided SQL query result and the question itself. Follow these steps to complete the task:

1. Carefully review the SQL query result provided. Understand the structure of the data, including column names and their corresponding values.
2. Read and understand the question that is being asked. Identify the specific information or insight the question is seeking from the SQL query result.
3. Cross-reference the SQL query result with the question to extract the relevant information. Ensure that your answer is accurate and directly addresses the question.
4. If the question requires calculations, aggregations, or logical reasoning based on the SQL query result, perform the necessary steps to derive the correct answer.
5. Provide a concise and clear answer to the question. Avoid including unnecessary details or information that is not relevant to the question.
6. Ensure that the output is in plain text and does not contain any XML tags or formatting. The response should be easy to read and understand. If the output needs to be displayed in a table format, use MD format for the table output.
7. Ask a follow-up question if the query does not contain enough information for you to answer the question.

Remember, your goal is to provide an accurate and contextually relevant answer based solely on the SQL query result and the question provided.

</instructions>

<examples>
<example>
<question>
What is the total sales amount for the month of January?
</question>
<sql_query_result>
| Month   | Sales_Amount |
|---------|--------------|
| January | 5000         |
| February| 7000         |
</sql_query_result>
<answer>
The total sales amount for the month of January is 5000.
</answer>
</example>

<example>
<question>
How many employees are in the IT department?
</question>
<sql_query_result>
| Department | Employee_Count |
|------------|----------------|
| IT         | 15             |
| HR         | 10             |
</sql_query_result>
<answer>
There are 15 employees in the IT department.
</answer>
</example>

<example>
<question>
What is the average score of students in the Math subject?
</question>
<sql_query_result>
| Subject | Average_Score |
|---------|---------------|
| Math    | 85            |
| Science | 90            |
</sql_query_result>
<answer>
The average score of students in the Math subject is 85.
</answer>
</example>
</examples>

Here is the question and query result:
<question>
"""+str(question)+"""
</question>
<sql_query_result>
"""+str(output_result)+"""
</sql_query_result>

```
"""

print(system_prompt)


```xml
<instructions>
You are tasked with answering a question based on the provided SQL query result and the question itself. Follow these steps to complete the task:

1. Carefully review the SQL query result provided. Understand the structure of the data, including column names and their corresponding values.
2. Read and understand the question that is being asked. Identify the specific information or insight the question is seeking from the SQL query result.
3. Cross-reference the SQL query result with the question to extract the relevant information. Ensure that your answer is accurate and directly addresses the question.
4. If the question requires calculations, aggregations, or logical reasoning based on the SQL query result, perform the necessary steps to derive the correct answer.
5. Provide a concise and clear answer to the question. Avoid including unnecessary details or information that is not relevant to the question.
6. Ensure that the output is in plain text and does not

In [150]:
import requests
import json

url = AZUREAI_ENDPOINT_URL

payload = json.dumps({
  "messages": [
      {
          "role": "system",
          "content": system_prompt
      }
  ],
  "frequency_penalty": 0,
  "presence_penalty": 0,
  "max_tokens": 4000,
  "seed": 42,
  "stop": "<|endoftext|>",
  "stream": False,
  "temperature": 0,
  "top_p": 1,
  "response_format": {
    "type": "text"
  }
})
headers = {
  'Content-Type': 'application/json',
  'api-key': AZUREAI_ENDPOINT_KEY
}

response = requests.request("POST", url, headers=headers, data=payload)
resp = json.loads(response.text)

print(resp['choices'][0]['message']['content'])
# print(resp)


The employee who sold the most products is Pamela Castillo.
